# Network Analysis

[Download relevant files here](https://melaniewalsh.org/Network-Analysis.zip)

In this lesson, we're going to learn about **network analysis**. Network analysis will help us better understand the complex relationships between groups of people, fictional characters, or other kinds of things.

# Import Libraries

To analyze network data, we're going to use the Python library [NetworkX](https://networkx.github.io/).

In [ ]:
!pip install networkx

In [ ]:
import networkx 

In [ ]:
import pandas as pd
pd.set_option('max_rows', 400)

In [ ]:
import matplotlib.pyplot as plt

# *Game of Thrones* Network

The network data that we're going to use in this lesson is taken from Andrew Beveridge and Jie Shan's paper, ["Network of Thrones."](https://www.maa.org/sites/default/files/pdf/Mathhorizons/NetworkofThrones%20%281%29.pdf) They calculated how many times each Game of Thrones character appeared within 15 words of another character in *A Storm of Swords*, the third book in the series.

For example:

> "It was the bastard **Jon Snow** who had taken that from him, him and his fat friend **Sam Tarly**."

> "Lucky it might be, and red it certainly was, but **Ygritte**’s hair was such a tangle that **Jon** was tempted to ask her if she only brushed it at the changing of the seasons."

> "**Arya** gave **Gendry** a sideways look. *He said it with me, like **Jon** used to do, back in Winterfell.* She missed **Jon Snow** the most of all her brothers.""

In [ ]:
got_df = pd.read_csv('../data/got-edges.csv')

In [ ]:
got_df

# Create a Network From a Pandas DataFrame

In [ ]:
G = networkx.from_pandas_edgelist(got_df, 'Source', 'Target', 'Weight')

# Output a Network File

In [ ]:
networkx.write_graphml(G, 'GOT-network.graphml')

# Draw a Simple Network

In [ ]:
networkx.draw(G)

In [ ]:
plt.figure(figsize=(8,8))
networkx.draw(G, with_labels=True, node_color='skyblue', width=.3, font_size=8)

# Calculate Degree

Who has the most number of connections in the network?

In [ ]:
networkx.degree(G)

Make the degree values a `dict`ionary, then add it as a network "attribute" with `networkx.set_node_attributes()`

In [ ]:
degrees = dict(networkx.degree(G))
networkx.set_node_attributes(G, name='degree', values=degrees)

Make a Pandas dataframe from the degree data `G.nodes(data='degree')`, then sort from highest to lowest

In [ ]:
degree_df = pd.DataFrame(G.nodes(data='degree'), columns=['node', 'degree'])
degree_df = degree_df.sort_values(by='degree', ascending=False)
degree_df

Plot the nodes with the highest degree values

In [ ]:
num_nodes_to_inspect = 10
degree_df[:num_nodes_to_inspect].plot(x='node', y='degree', kind='barh').invert_yaxis()

# Calculate Weighted Degree

Who has the most number of connections in the network (if you factor in edge weight)?

In [ ]:
networkx.degree(G, weight='Weight')

Make the weighted degree values a `dict`ionary, then add it as a network "attribute" with `networkx.set_node_attributes()`

In [ ]:
weighted_degrees = dict(networkx.degree(G, weight='Weight'))
networkx.set_node_attributes(G, name='weighted_degree', values=weighted_degrees)

Make a Pandas dataframe from the degree data `G.nodes(data='weighted_degree')`, then sort from highest to lowest

In [ ]:
weighted_degree_df = pd.DataFrame(G.nodes(data='weighted_degree'), columns=['node', 'weighted_degree'])
weighted_degree_df = weighted_degree_df.sort_values(by='weighted_degree', ascending=False)
weighted_degree_df

Plot the nodes with the highest weighted degree values

In [ ]:
num_nodes_to_inspect = 10
weighted_degree_df[:num_nodes_to_inspect].plot(x='node', y='weighted_degree', color='orange', kind='barh').invert_yaxis()

# Calculate Betweenness Centrality Scores

Who connects the most other nodes in the network?

In [ ]:
networkx.betweenness_centrality(G)

In [ ]:
betweenness_centrality = networkx.betweenness_centrality(G)

Add `betweenness_centrality` (which is already a dictionary) as a network "attribute" with `networkx.set_node_attributes()`

In [ ]:
networkx.set_node_attributes(G, name='betweenness', values=betweenness_centrality)

Make a Pandas dataframe from the betweenness data `G.nodes(data='betweenness')`, then sort from highest to lowest

In [ ]:
betweenness_df = pd.DataFrame(G.nodes(data='betweenness'), columns=['node', 'betweenness'])
betweenness_df = betweenness_df.sort_values(by='betweenness', ascending=False)
betweenness_df

Plot the nodes with the highest betweenness centrality scores

In [ ]:
num_nodes_to_inspect = 10
betweenness_df[:num_nodes_to_inspect].plot(x='node', y='betweenness', color='green', kind='barh').invert_yaxis()

# Communities

Who forms distinct communities within this network?

In [ ]:
from networkx.algorithms import community

Calculate communities with `community.greedy_modularity_communities()`

In [ ]:
communities = community.greedy_modularity_communities(G)

In [ ]:
communities

Make a `dict`ionary by looping through the communities and, for each member of the community, adding their community number

In [ ]:
# Create empty dictionary
modularity_class = {}
#Loop through each community in the network
for community_number, community in enumerate(communities):
    #For each member of the community, add their community number
    for name in community:
        modularity_class[name] = community_number

Add modularity class to the network as an attribute

In [ ]:
networkx.set_node_attributes(G, modularity_class, 'modularity_class')

Make a Pandas dataframe from modularity class network data `G.nodes(data='modularity_class')`

In [ ]:
communities_df = pd.DataFrame(G.nodes(data='modularity_class'), columns=['node', 'modularity_class'])
communities_df = communities_df.sort_values(by='modularity_class', ascending=False)

In [ ]:
communities_df

Inspect each community in the network

In [ ]:
communities_df[communities_df['modularity_class'] == 4]

In [ ]:
communities_df[communities_df['modularity_class'] == 3]

In [ ]:
communities_df[communities_df['modularity_class'] == 2]

In [ ]:
communities_df[communities_df['modularity_class'] == 1]

In [ ]:
communities_df[communities_df['modularity_class'] == 0]

Plot a sample of 50 characters with their modularity class indicated by a star

In [ ]:
fig, ax = plt.subplots()
communities_df.sample(50).plot(x='modularity_class', y='node', c='modularity_class', colormap='viridis',
                               kind='scatter', s=100, marker='*', figsize=(5,10), legend=True, ax=ax)

# All Network Metrics

Create a Pandas dataframe of all network attributes by creating a `dict`ionary of `G.nodes(data=True)`...

In [ ]:
dict(G.nodes(data=True))

...and then [transposing it](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.T.html) (flipping the columns and rows) with `.T`

In [ ]:
nodes_df = pd.DataFrame(dict(G.nodes(data=True))).T
nodes_df

In [ ]:
nodes_df.sort_values(by='betweenness', ascending=False)